In [1]:
import numpy

In [2]:
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
print(tf.__version__)

2025-07-16 16:18:21.616050: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.19.0


In [4]:
inputfeaturesL=numpy.load('/home/TrentonBuckley/Documents/R&LHandNN/Labels/Input/inputfeaturesL.npy')

In [5]:
inputfeaturesR=numpy.load('/home/TrentonBuckley/Documents/R&LHandNN/Labels/Input/inputfeaturesR.npy')

In [6]:
outputlabelsL=numpy.load('/home/TrentonBuckley/Documents/R&LHandNN/Labels/Output/outputlabelsL.npy')

In [7]:
outputlabelsR=numpy.load('/home/TrentonBuckley/Documents/R&LHandNN/Labels/Output/outputlabelsR.npy')

In [8]:
print(inputfeaturesL.shape)

(10000, 8)


In [9]:
inputfeaturesLR=numpy.concatenate([inputfeaturesL, inputfeaturesR], axis=0)

In [10]:
outputlabelsLR=numpy.concatenate([outputlabelsL, outputlabelsR], axis=0)

In [11]:
p=numpy.random.permutation(len(inputfeaturesLR))

In [12]:
inputfeaturesLR_shuffled=inputfeaturesLR[p]

In [13]:
outputlabelsLR_shuffled=outputlabelsLR[p]

In [14]:
inputfeatures_test_shuffled=inputfeaturesLR_shuffled[0:5000]

In [15]:
inputfeatures_training_shuffled=inputfeaturesLR_shuffled[-15000:-1]

In [16]:
outputlabels_test_shuffled=outputlabelsLR_shuffled[0:5000]

In [17]:
outputlabels_training_shuffled=outputlabelsLR_shuffled[-15000:-1]

In [18]:
dataset_training=tf.data.Dataset.from_tensor_slices((inputfeatures_training_shuffled, outputlabels_training_shuffled))

In [19]:
dataset_test=tf.data.Dataset.from_tensor_slices((inputfeatures_test_shuffled, outputlabels_test_shuffled))

In [20]:
dataset_training.save("/home/TrentonBuckley/Documents/R&LHandNN/TensorFlowFiles/dataset_training.tf")

In [21]:
dataset_test.save("/home/TrentonBuckley/Documents/R&LHandNN/TensorFlowFiles/dataset_test.tf")